Importing all required modules

In [1]:
import geopandas as gpd
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import altair as alt
from matplotlib import pyplot as plt
import numpy as np
from altair import datum
import streamlit as st
import datetime

import altair_saver

In [2]:
import altair_viewer

Reading the three files required fo this project

In [3]:
source1 = './Data/Raw/NYPD_Shooting_Incident_Data__Historic.csv'
source2 = './Data/Raw/New_York_City_Population.csv'
source3= './Data/Raw/NYC Borough Boundaries.geojson'

shooting = pd.read_csv(source1)
boro_pop = pd.read_csv(source2)
boro_boundary = gpd.read_file(source3)

**Quick review and pre-processing of the _shooting_ dataset**

In [4]:
shooting.head(5)

,INCIDENT_KEY,OCCUR_DATE,OCCUR_TIME,BORO,PRECINCT,JURISDICTION_CODE,LOCATION_DESC,STATISTICAL_MURDER_FLAG,PERP_AGE_GROUP,PERP_SEX,PERP_RACE,VIC_AGE_GROUP,VIC_SEX,VIC_RACE,X_COORD_CD,Y_COORD_CD,Latitude,Longitude,Lon_Lat
0,236168668,11/11/2021,15:04:00,BROOKLYN,79,0.0,NaN,False,NaN,NaN,NaN,18-24,M,BLACK,996313.0,187499.0,40.681318,-73.956509,POINT (-73.95650899099996 40.68131820000008)
1,231008085,07/16/2021,22:05:00,BROOKLYN,72,0.0,NaN,False,45-64,M,ASIAN / PACIFIC ISLANDER,25-44,M,ASIAN / PACIFIC ISLANDER,981845.0,171118.0,40.636364,-74.008667,POINT (-74.00866668999998 40.63636384100005)
2,230717903,07/11/2021,01:09:00,BROOKLYN,79,0.0,NaN,False,<18,M,BLACK,25-44,M,BLACK,996546.0,187436.0,40.681145,-73.955669,POINT (-73.95566903799994 40.68114495900005)
3,237712309,12/11/2021,13:42:00,BROOKLYN,81,0.0,NaN,False,NaN,NaN,NaN,25-44,M,BLACK,1001139.0,192775.0,40.695792,-73.939096,POINT (-73.939095905 40.69579171600003)
4,224465521,02/16/2021,20:00:00,QUEENS,113,0.0,NaN,False,NaN,NaN,NaN,25-44,M,BLACK,1050710.0,184826.0,40.673740,-73.760411,POINT (-73.76041066999993 40.67374017600008)


In [5]:
shooting.columns

Index(['INCIDENT_KEY', 'OCCUR_DATE', 'OCCUR_TIME', 'BORO', 'PRECINCT',
       'JURISDICTION_CODE', 'LOCATION_DESC', 'STATISTICAL_MURDER_FLAG',
       'PERP_AGE_GROUP', 'PERP_SEX', 'PERP_RACE', 'VIC_AGE_GROUP', 'VIC_SEX',
       'VIC_RACE', 'X_COORD_CD', 'Y_COORD_CD', 'Latitude', 'Longitude',
       'Lon_Lat'],
      dtype='object')

In [6]:
# dropping unrequired columns
shooting= shooting.drop(['PRECINCT','JURISDICTION_CODE','LOCATION_DESC','X_COORD_CD','Y_COORD_CD','Lon_Lat', 'LOCATION_DESC'], axis=1)

In [7]:
# changing all column names to lowercase and renaming the statistical_murder_flag... 
# column for convenience/ personal preference
shooting.columns=shooting.columns.str.lower()
shooting = shooting.rename(columns={'statistical_murder_flag': 'murder_flag'})

In [8]:
# First we will view general information about the dataset in order to know the different...
# columns and their respective data types.
shooting.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25596 entries, 0 to 25595
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   incident_key    25596 non-null  int64  
 1   occur_date      25596 non-null  object 
 2   occur_time      25596 non-null  object 
 3   boro            25596 non-null  object 
 4   murder_flag     25596 non-null  bool   
 5   perp_age_group  16252 non-null  object 
 6   perp_sex        16286 non-null  object 
 7   perp_race       16286 non-null  object 
 8   vic_age_group   25596 non-null  object 
 9   vic_sex         25596 non-null  object 
 10  vic_race        25596 non-null  object 
 11  latitude        25596 non-null  float64
 12  longitude       25596 non-null  float64
dtypes: bool(1), float64(2), int64(1), object(9)
memory usage: 2.4+ MB


In [9]:
# Next, we will convert the _occur_date_ and _occur_time_ columns to datetime format
shooting['occur_date'] = pd.to_datetime(shooting.occur_date)
shooting['occur_time'] = pd.to_datetime(shooting.occur_time)

In [10]:
# Now, we check for null values in the dataset
len(shooting), len(shooting[shooting.isnull().any(axis=1)])


(25596, 9344)

From the result above, there are a total of 25,596 rows in the shooting dataframe. 9,344 rows have NA in one or more cells.

In the next step, we will  use a for- loop to check for the number of empty cells for each column the _shooting_ dataframe

In [11]:
# We check for the number of null cellss in each column of the data
cols = list(shooting.columns)


col_null_count = []
for col in cols:
    col_null_count.append( len(shooting[shooting[col].isna()]))


pd.DataFrame(list(zip(cols, col_null_count)), columns =['col_name', 'null_count'])

,col_name,null_count
0,incident_key,0
1,occur_date,0
2,occur_time,0
3,boro,0
4,murder_flag,0
5,perp_age_group,9344
6,perp_sex,9310
7,perp_race,9310
8,vic_age_group,0
9,vic_sex,0


From result above, we can see that there are 9344, 9310 and 9310 null instances in the columns perp_age_group, perp_sex, perp_race respectively. These are the instances where the perpetrator of a shooting is unknown and has not been apprehended by the security forces. We will keep this in mind as we proceed with the project.

**Quick review and pre-processing of the _boro_pop_ dataset**

In [12]:
boro_pop

,Age Group,Borough,1950,1950 - Boro share of NYC total,1960,1960 - Boro share of NYC total,1970,1970 - Boro share of NYC total,1980,1980 - Boro share of NYC total,...,2000,2000 - Boro share of NYC total,2010,2010 - Boro share of NYC total,2020,2020 - Boro share of NYC total,2030,2030 - Boro share of NYC total,2040,2040 - Boro share of NYC total
0,Total Population,NYC Total,7891957,100.00,7781984,100.00,7894862,100.00,7071639,100.00,...,8008278,100.00,8242624,100.00,8550971,100.00,8821027,100.00,9025145,100.00
1,Total Population,Bronx,1451277,18.39,1424815,18.31,1471701,18.64,1168972,16.53,...,1332650,16.64,1385108,16.80,1446788,16.92,1518998,17.22,1579245,17.50
2,Total Population,Brooklyn,2738175,34.70,2627319,33.76,2602012,32.96,2230936,31.55,...,2465326,30.78,2552911,30.97,2648452,30.97,2754009,31.22,2840525,31.47
3,Total Population,Manhattan,1960101,24.84,1698281,21.82,1539233,19.50,1428285,20.20,...,1537195,19.20,1585873,19.24,1638281,19.16,1676720,19.01,1691617,18.74
4,Total Population,Queens,1550849,19.65,1809578,23.25,1986473,25.16,1891325,26.75,...,2229379,27.84,2250002,27.30,2330295,27.25,2373551,26.91,2412649,26.73
5,Total Population,Staten Island,191555,2.43,221991,2.85,295443,3.74,352121,4.98,...,443728,5.54,468730,5.69,487155,5.70,497749,5.64,501109,5.55


In [13]:
# Extracting only the required columns from the boro_pop dataset
boro_pop = boro_pop[['Borough','2020']]

In [14]:
# renaming the columns for uniformity and easy understanding. This will be vital for merge operation which 
# will be performed later in this project
boro_pop= boro_pop.rename(columns={'Borough':'boro', '2020':'population'})

In [15]:
# stripping leading and trailing whitespaces from the records in the boro column. From previous handling of the...
# dataset, it was discovered that there are leading white spaces in the boro column, leading to incorrect...
# result of PD Merge (which will  be performed later in this project)
boro_pop['boro'] = boro_pop['boro'].str.strip()
boro_pop['boro'] = boro_pop['boro'].str.upper()
boro_pop

,boro,population
0,NYC TOTAL,8550971
1,BRONX,1446788
2,BROOKLYN,2648452
3,MANHATTAN,1638281
4,QUEENS,2330295
5,STATEN ISLAND,487155


**Quick review and pre-processing of the _boro_boundary_ dataset**

In [16]:
boro_boundary

,boro_code,boro_name,shape_area,shape_leng,geometry
0,5,Staten Island,1623620725.06,325917.353702,"MULTIPOLYGON (((-74.05051 40.56642, -74.05047 ..."
1,2,Bronx,1187182350.92,463176.004334,"MULTIPOLYGON (((-73.89681 40.79581, -73.89694 ..."
2,3,Brooklyn,1934229471.99,728263.543413,"MULTIPOLYGON (((-73.86327 40.58388, -73.86381 ..."
3,1,Manhattan,636520830.696,357564.317228,"MULTIPOLYGON (((-74.01093 40.68449, -74.01193 ..."
4,4,Queens,3041418543.49,888199.780587,"MULTIPOLYGON (((-73.82645 40.59053, -73.82642 ..."


In [17]:
# renaming the columns for uniformity and easy understanding and changing the values in that colun to uppercase.
# These steps are vital for the success of the merge operation which will be performed later in this project
 
boro_boundary = boro_boundary.rename(columns={'boro_name': 'boro'})
boro_boundary['boro'] = boro_boundary['boro'].str.upper()
boro_boundary

,boro_code,boro,shape_area,shape_leng,geometry
0,5,STATEN ISLAND,1623620725.06,325917.353702,"MULTIPOLYGON (((-74.05051 40.56642, -74.05047 ..."
1,2,BRONX,1187182350.92,463176.004334,"MULTIPOLYGON (((-73.89681 40.79581, -73.89694 ..."
2,3,BROOKLYN,1934229471.99,728263.543413,"MULTIPOLYGON (((-73.86327 40.58388, -73.86381 ..."
3,1,MANHATTAN,636520830.696,357564.317228,"MULTIPOLYGON (((-74.01093 40.68449, -74.01193 ..."
4,4,QUEENS,3041418543.49,888199.780587,"MULTIPOLYGON (((-73.82645 40.59053, -73.82642 ..."


In [18]:
# Creating 'year' column in the 'shooting' dataset by extracting the year of occurence from occur_date column
shooting['year']= shooting["occur_date"].dt.year

In [19]:
# Adding a column with the value 'TOTAL' . This will be useful for later visualisations
shooting['total']= 'TOTAL'

Writing our cleaned/pre-processed data to file

After reviewing and pre-processing the datasets , we can now explore and visualise the data

In [20]:
shooting.to_pickle('./Data/Cleaned/shooting.pkl')
boro_pop.to_pickle('./Data/Cleaned/boro_pop.pkl')
boro_boundary.to_pickle('./Data/Cleaned/boro_boundary.pkl')

In [27]:
chart = alt.Chart(boro_pop).mark_bar(tooltip= True).encode(
    x= alt.X("boro:N"),
    y=alt.Y("count(population)"))

alt.renderers.enable('mimetype')
chart



<VegaLite 4 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/troubleshooting.html


In [22]:
alt.renderers

chart

<VegaLite 4 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/troubleshooting.html
